## Text Summarization
#### Ming Gong 

## Answers : 
  ### a. How I approach? 
  #### First, I did the tokenize and data cleaning which include 'Remove all the references and extra spaces'
  ####  'Convert into Lower cases, remove all punctuation, digits', and break all big text into sentences using tokenize
  #### Second, I calculate the word score by counting the word relatively frequency. I use the word frequency divided by the Max frequently word in the paragraph. Thus, all the word frequency are bwteen 0 and 1. 
  #### Then, I do back to the sentence level, to calculate the sentence score by adding up the word frequency score for all the words in a srntence. Finally, pick out the highest score word to be the summarize sentence for the text. 


  ### b. How I evaluate? 
#### In general:

#### Bleu measures precision: how much the words (and/or n-grams) in the machine generated summaries appeared in the human reference summaries.

#### Rouge measures recall: how much the words (and/or n-grams) in the human reference summaries appeared in the machine generated summaries.

#### you could use the F1 measure to make the metrics work together: F1 = 2 * (Bleu * Rouge) / (Bleu + Rouge)
#### F1 score: the highest, the better. 

In [150]:
## Import Packages

import pandas as pd
import re
import nltk
import heapq
import tqdm
import numpy as np

In [27]:
## Import Dataset 

df=pd.read_csv('/Users/minggong/Desktop/un.csv.')

In [28]:
## Have a first glance of the dataset

df.head()

,session,year,country,text
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ..."
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ..."
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...
4,44,1989,ZWE,﻿I should like at the outset to express my del...


In [75]:
# Preprocessing the text
##  Data/Text Cleaning

text = df.text.apply(lambda text: re.sub(r'\[[0-9]*\]',' ', text))## Move Reference
text = df.text.apply(lambda text: re.sub(r'\s+',' ',text))## Move extra space

clean_text = df.text.apply(lambda clean_text: clean_text.lower()) ## Convert to a lower case 

## Remove all extra punctuation, digits, extra spaces 
clean_text = df.text.apply(lambda clean_text: re.sub(r'\W',' ',clean_text))
clean_text = df.text.apply(lambda clean_text: re.sub(r'\d',' ',clean_text))
clean_text = df.text.apply(lambda clean_text: re.sub(r'\s+',' ',clean_text))


In [81]:
## Separate  the pararaph into sentences 

sentences = text.apply(lambda text: nltk.sent_tokenize(text))

stop_words = nltk.corpus.stopwords.words('english')

print(sentences)

0       [﻿It is indeed a pleasure for me and the membe...
1       [﻿ May I begin by congratulating you., Sir, on...
2       [﻿ Mr. President, it is a particular pleasure ...
3       [﻿ During the debate at the fortieth session o...
4       [﻿I should like at the outset to express my de...
                              ...                        
7502    [﻿This session that is taking place under extr...
7503    [﻿I am honoured to participate in this histori...
7504    [﻿It is for me a signal honour to take the flo...
7505    [﻿First, may I congratulate Mr. Han Seung-soo ...
7506    [﻿On behalf of the State of Kuwait, it gives m...
Name: text, Length: 7507, dtype: object


In [87]:
## Sentences to words 

a = clean_text.apply(lambda t: nltk.word_tokenize(t))

0       [﻿It, is, indeed, a, pleasure, for, me, and, t...
1       [﻿, May, I, begin, by, congratulating, you, .,...
2       [﻿, Mr., President, ,, it, is, a, particular, ...
3       [﻿, During, the, debate, at, the, fortieth, se...
4       [﻿I, should, like, at, the, outset, to, expres...
                              ...                        
7502    [﻿This, session, that, is, taking, place, unde...
7503    [﻿I, am, honoured, to, participate, in, this, ...
7504    [﻿It, is, for, me, a, signal, honour, to, take...
7505    [﻿First, ,, may, I, congratulate, Mr., Han, Se...
7506    [﻿On, behalf, of, the, State, of, Kuwait, ,, i...
Name: text, Length: 7507, dtype: object

In [101]:
## Calculate the word frequency score 

new_col = []
for para in a:
    word2count = {} 
    for word in para:
        if word not in stop_words:                  
            if word not in word2count.keys():
                word2count[word]=1
            else:
                word2count[word]+=1
    for key in word2count.keys():                   
        word2count[key]=word2count[key]/max(word2count.values())
    new_col.append(word2count)

## Add dictionaries as new columns to the dataset

df['clean_text'] = clean_text
df['para_score'] = new_col
df['sentence'] = sentences

In [144]:
## Calculate the summary of the word score in the sentences 

lst_of_summaries = []
for row in df.iterrows():
    score = row[1]['para_score']
    lst_sent_score = []
    for sent in row[1]['sentence']:
        words = nltk.word_tokenize(sent)
        sent_score = 0
        for word in words:
            if word in score.keys():
                sent_score += score[word]
        lst_sent_score.append(sent_score)
    #print(lst_sent_score)
    lst_of_summaries.append(row[1]['sentence'][np.argmax(lst_sent_score)])
print(lst_of_summaries)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [145]:
## Add the New Summary column to the dataset 

df['summary'] = lst_of_summaries
print(df.head())

   session  year country                                               text  \
0       44  1989     MDV  ﻿It is indeed a pleasure for me and the member...   
1       44  1989     FIN  ﻿\nMay I begin by congratulating you. Sir, on ...   
2       44  1989     NER  ﻿\nMr. President, it is a particular pleasure ...   
3       44  1989     URY  ﻿\nDuring the debate at the fortieth session o...   
4       44  1989     ZWE  ﻿I should like at the outset to express my del...   

                                          clean_text  \
0  ﻿It is indeed a pleasure for me and the member...   
1  ﻿ May I begin by congratulating you. Sir, on y...   
2  ﻿ Mr. President, it is a particular pleasure f...   
3  ﻿ During the debate at the fortieth session of...   
4  ﻿I should like at the outset to express my del...   

                                          para_score  \
0  {'﻿It': 0.008264462809917356, 'indeed': 0.0165...   
1  {'﻿': 0.007874015748031496, 'May': 0.015748031...   
2  {'﻿': 0.003597122

In [149]:
df.head(20)

,session,year,country,text,clean_text,para_score,sentence,summary
0,44,1989,MDV,﻿It is indeed a pleasure for me and the member...,﻿It is indeed a pleasure for me and the member...,"{'﻿It': 0.008264462809917356, 'indeed': 0.0165...",[﻿It is indeed a pleasure for me and the membe...,The implications that efforts to strengthen th...
1,44,1989,FIN,"﻿\nMay I begin by congratulating you. Sir, on ...","﻿ May I begin by congratulating you. Sir, on y...","{'﻿': 0.007874015748031496, 'May': 0.015748031...","[﻿ May I begin by congratulating you., Sir, on...",If such elections are properly mentored and ta...
2,44,1989,NER,"﻿\nMr. President, it is a particular pleasure ...","﻿ Mr. President, it is a particular pleasure f...","{'﻿': 0.0035971223021582736, 'Mr.': 0.02158273...","[﻿ Mr. President, it is a particular pleasure ...",We call on it firmly to support the Gbadolite ...
3,44,1989,URY,﻿\nDuring the debate at the fortieth session o...,﻿ During the debate at the fortieth session of...,"{'﻿': 0.007462686567164179, 'During': 0.007462...",[﻿ During the debate at the fortieth session o...,The cessation of hostilities between Iran and ...
4,44,1989,ZWE,﻿I should like at the outset to express my del...,﻿I should like at the outset to express my del...,"{'﻿I': 0.005555555555555556, 'like': 0.0111111...",[﻿I should like at the outset to express my de...,"It was in recognition of that fact, and motiva..."
5,44,1989,PHL,"﻿Before you began to occupy that exalted seat,...","﻿Before you began to occupy that exalted seat,...","{'﻿Before': 0.007692307692307693, 'began': 0.0...",[﻿Before you began to occupy that exalted seat...,Is not this the moment for the debtors to exte...
6,44,1989,SDN,﻿It gives me great pleasure to congratulate Am...,﻿It gives me great pleasure to congratulate Am...,"{'﻿It': 0.009615384615384616, 'gives': 0.00961...",[﻿It gives me great pleasure to congratulate A...,Since the General Assembly met last year in it...
7,44,1989,RUS,﻿My task as head of the delegation of the Sovi...,﻿My task as head of the delegation of the Sovi...,"{'﻿My': 0.004329004329004329, 'task': 0.004329...",[﻿My task as head of the delegation of the Sov...,"The Soviet Union is ready, together with the U..."
8,44,1989,CHN,﻿\nPermit me to begin by warmly congratulating...,﻿ Permit me to begin by warmly congratulating ...,"{'﻿': 0.006134969325153374, 'Permit': 0.006134...",[﻿ Permit me to begin by warmly congratulating...,"On the one hand, there are vast numbers of cou..."
9,44,1989,ESP,﻿I should like to express my sincere congratul...,﻿I should like to express my sincere congratul...,"{'﻿I': 0.0049261083743842365, 'like': 0.049261...",[﻿I should like to express my sincere congratu...,Work in this direction must proceed along with...
